In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange='mufex',
    symbol='BTCUSDT',
    timeframe='5m',
    since_datetime=datetime(2024,2,14,9,20),
    until_datetime=datetime(2024,2,25),
)
datetimes = candles[:, CandleBodyType.Timestamp].astype("datetime64[ms]")
open_prices = candles[:, CandleBodyType.Open]
high_prices = candles[:, CandleBodyType.High]
low_prices = candles[:, CandleBodyType.Low]
closing_prices = candles[:, CandleBodyType.Close]

In [3]:
candles

array([[1707902400000.00, 51244.00, 51321.50, 51165.60, 51271.40, 237.25],
       [1707902700000.00, 51271.40, 51301.60, 51155.70, 51291.10, 175.12],
       [1707903000000.00, 51291.10, 51311.80, 51168.50, 51252.30, 140.91],
       ...,
       [1708818300000.00, 51615.40, 51615.40, 51581.00, 51585.20, 21.83],
       [1708818600000.00, 51585.20, 51586.80, 51576.00, 51579.60, 5.00],
       [1708818900000.00, 51579.60, 51609.90, 51579.20, 51602.20, 15.75]])

In [4]:
open_prices

array([51244.00, 51271.40, 51291.10, ..., 51615.40, 51585.20, 51579.60])

In [3]:
price_pct = .0075

In [4]:
closing_prices[0] + (closing_prices[0] * price_pct)

51655.9355

In [5]:
closing_prices[0] - (closing_prices[0] * price_pct)

50886.8645

In [6]:
sell_order = closing_prices[0] + (closing_prices[0] * price_pct)
sell_signals = np.full_like(closing_prices, np.nan)

buy_order = closing_prices[0] - (closing_prices[0] * price_pct)
buy_signals = np.full_like(closing_prices, np.nan)

In [7]:
for idx in range(1,closing_prices.size):
    if low_prices[idx] <= buy_order:
        buy_signals[idx] = buy_order
        buy_order = closing_prices[idx] - (closing_prices[idx] * price_pct)
        sell_order = closing_prices[idx] + (closing_prices[idx] * price_pct)
    elif high_prices[idx] >= sell_order:
        sell_signals[idx] = sell_order
        buy_order = closing_prices[idx] - (closing_prices[idx] * price_pct)
        sell_order = closing_prices[idx] + (closing_prices[idx] * price_pct)

average entry

(usdt 1 + usdt 2) / (qty 1 + qty 2)

In [8]:
(100 + 100) / ((100 / 51708.73) + (100 / 52004.03))

51855.95959806489

In [9]:
go.Figure(
    data=[
        go.Candlestick(
            x=datetimes,
            open=open_prices,
            high=high_prices,
            low=low_prices,
            close=closing_prices,
            name="Candles",
        ),
        go.Scatter(
            x=datetimes,
            y=buy_signals,
            mode="markers",
            name="Buy Signals",
            marker=dict(
                size=15,
                symbol="circle",
                color="#00F6FF",
                line=dict(
                    width=1,
                    color="DarkSlateGrey",
                ),
            ),
        ),
        go.Scatter(
            x=datetimes,
            y=sell_signals,
            mode="markers",
            name="Sell Signals",
            marker=dict(
                size=15,
                symbol="circle",
                color="yellow",
                line=dict(
                    width=1,
                    color="DarkSlateGrey",
                ),
            ),
        ),
    ]
).update_layout(height=800, xaxis_rangeslider_visible=False)